# Extract Features

1. Read csv files and load as dfs
2. Combine dfs
3. Get POS, NER
4. Save in new features_df
5. Get semantic cosine similarity

In [1]:
import os, sys

import pandas as pd

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from log_files import LogData
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
df = log_files.read_data(notebook_dir, log_file_path, predictions)
df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs/batch_1-predictions
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs/batch_1-predictions/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs/batch_2-predictions
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness_experiments/../data/prediction_logs/batch_2-predictions/batch_2-from_df.csv
save_batch_directory: /U

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,"On August 21, 2024, as a financial analyst, I predict that the revenue at Amazon will potentially decrease in Q3 of 2027.",1,finance,gpt-3.5-turbo,NAVI_GATOR,0,1
1,"On 2024-08-21, Morgan Stanley speculates the operating income at Amazon will likely increase.",1,finance,gpt-4o,NAVI_GATOR,0,2
2,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,1
3,"On August 21, 2023, Goldman Sachs speculates that the stock price at Amazon will likely increase.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,2
4,"George, a financial analyst, predicts that on 01/15/2024, the Google revenue may rise.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,3
5,"According to BlackRock, the operating cash flow at ExxonMobil would fall in Q3 2023.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,4
6,"In Q2 2023, Wells Fargo envisions that the stock price at Tesla has some probability to remain stable.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,5


In [ ]:
df∫

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,"On August 21, 2024, as a financial analyst, I predict that the revenue at Amazon will potentially decrease in Q3 of 2027.",1,finance,gpt-3.5-turbo,NAVI_GATOR,0,1
1,"On 2024-08-21, Morgan Stanley speculates the operating income at Amazon will likely increase.",1,finance,gpt-4o,NAVI_GATOR,0,2
2,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,1
3,"On August 21, 2023, Goldman Sachs speculates that the stock price at Amazon will likely increase.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,2
4,"George, a financial analyst, predicts that on 01/15/2024, the Google revenue may rise.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,3
5,"According to BlackRock, the operating cash flow at ExxonMobil would fall in Q3 2023.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,4
6,"In Q2 2023, Wells Fargo envisions that the stock price at Tesla has some probability to remain stable.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,5
7,"The stock price at Apple should stay the same in August 2023, according to Morgan Stanley.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,6
8,Dr. Smith predicts that the prevalence of chronic illnesses at rural health clinics will potentially decrease in Q4 of 2026.,1,health,gpt-3.5-turbo,NAVI_GATOR,0,1
9,WHO forecasts that the prevalence of chronic illnesses at global urban centers potentially decrease in Q4 of 2024.,1,health,gpt-4o,NAVI_GATOR,0,1


In [ ]:
df[2 : 3]

- issue: `obesity rate` is not being captures by POS or NER.

In [ ]:
only_predictions = DataProcessing.df_to_list(df, 'Base Sentence')
print(only_predictions)

sfe_class = SpacyFeatureExtraction(df, "Base Sentence")
word_leveL_disable_components = ["lemmatizer"]
word_level_pos_tags, word_level_pos_mappings, word_level_ner_entities, word_level_ner_mappings =  sfe_class.extract_features(data=only_predictions, disable_components=word_leveL_disable_components)
all_word_level_pos_df = DataProcessing.convert_to_df(word_level_pos_tags, mapping=word_level_pos_mappings)
all_word_level_ner_df = DataProcessing.convert_to_df(word_level_ner_entities, word_level_ner_mappings)
pred_pos_ner_features = [all_word_level_pos_df, all_word_level_ner_df]
pred_pos_ner_features_df = DataProcessing.concat_dfs(pred_pos_ner_features, axis=1, ignore_index=False)
pred_pos_ner_features_df.head()

## Observations

In [ ]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

In [ ]:
only_observations = DataProcessing.df_to_list(observations_df, 'Base Sentence')
# print(only_observations)
word_level_pos_tags, word_level_pos_mappings, word_level_ner_entities, word_level_ner_mappings = sfe_class.extract_features(data=only_observations, disable_components=word_leveL_disable_components)
all_word_level_pos_df = DataProcessing.convert_to_df(word_level_pos_tags, mapping=word_level_pos_mappings)
all_word_level_ner_df = DataProcessing.convert_to_df(word_level_ner_entities, word_level_ner_mappings)
obs_pos_ner_features = [all_word_level_pos_df, all_word_level_ner_df]
obs_pos_ner_features_df = DataProcessing.concat_dfs(obs_pos_ner_features, axis=1, ignore_index=False)
obs_pos_ner_features_df.head(3)

## Both

- Create a knowledge graph
    - Nodes: words
    - Edges: connection to other words (same/diff sentence)
- Look at code from Graphbreeding project on 2019 Mac

In [ ]:
pred_sentence_features_df = DataProcessing.include_sentence_and_label(pred_pos_ner_features_df, df)
pred_sentence_features_df.head(3)
pred_profit_filt = (pred_pos_ner_features_df.NOUN_1.values == 'profit')
pred_profit_df = pred_pos_ner_features_df.loc[pred_profit_filt == True]
pred_profit_df

In [ ]:
pred_profit_sentences = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_2 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_3 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_4 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')


In [ ]:
obs_profit_sentence_df = DataProcessing.include_sentence_and_label(obs_pos_ner_features_df, observations_df)
obs_profit_filt = (obs_pos_ner_features_df.NOUN_1.values == 'profit')
obs_profit_df = obs_pos_ner_features_df.loc[obs_profit_filt == True]
obs_profit_df

In [ ]:
obs_profit_sentences = DataProcessing.df_to_list(obs_profit_df, 'Base Sentence')
obs_profit_sentences

In [ ]:
pred_obs_dict = {}
for pred_sent_idx in range(len(pred_profit_sentences)):
    pred_sentence = pred_profit_sentences[pred_sent_idx]
    pred_obs_dict[pred_sentence] = obs_profit_sentences
pred_obs_dict

In [ ]:
import spacy

load_nlp_model = spacy.load("en_core_web_sm")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in pred_obs_dict.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in pred_obs_dict[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings

# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df['Scores'] = "Similarity"
df

In [ ]:
data

In [ ]:
table = pd.pivot_table(df, values='Spacy Similarity', index=['Prediction', 'Observation'],
                       columns=['Scores'])

- Only grouping by `p_a = profit`. 
- What if also group by sentiment?
    - fall/decrease ---  drastically different --- neutral bc dd doesn't indicate slope/how
    - fall --- plummeted, so similar

In [ ]:
table

In [ ]:
# Methods to alter Precision and Recall for prob rather than binary
    
#     https://en.wikipedia.org/wiki/Fuzzy_classification

# Support or deny classes

In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

In [ ]:
pred_profit_sentences

In [ ]:
obs_profit_sentences

In [ ]:
my_sentences = pred_profit_sentences + obs_profit_sentences
my_sentences

In [ ]:
my_sentences = ["10%", "10.5", "11"]

In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(my_sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])